In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split as TTS
import itertools
from progressbar import progressbar as pb

data = pd.read_csv('/content/drive/My Drive/Embeddings/big_data_labels.csv')
data.head()

,0,1,2,3,4,5,label
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2,История
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2,История
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3,История
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1,История
4,Творчество поэтов Гумилева Ахматовой Мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1,История


In [0]:
import pickle

with open('/content/drive/My Drive/Embeddings/elmo_embed_big.pickle', 'rb') as f:
    elmo_embs = pickle.load(f)

elmo_embs = np.array(elmo_embs)

with open('/content/drive/My Drive/Embeddings/bert_embed_big.pickle', 'rb') as f:
    bert_embs = pickle.load(f)

bert_embs = np.array(bert_embs)

with open('/content/drive/My Drive/Embeddings/fasttext_embed_big.pickle', 'rb') as f:
    fasttext_embs = pickle.load(f)

fasttext_embs = np.array(fasttext_embs)

with open('/content/drive/My Drive/Embeddings/rusvec_embed_big.pickle', 'rb') as f:
    rusvec_embs = pickle.load(f)

rusvec_embs = np.array(rusvec_embs)

In [0]:
def get_batch(X_val, y_val, batch_sz=16):
    
    '''
    Разбиение выборки на батчи заданного размера.
    
    X_val - векторное представление данных
    y_val - разметка
    batch_sz - размер одного батча
    shuffle - перемешивание элементов по батчам
    
    return:
    
    X_batches, y_batches - разделенная по батчам выборка    
    
    '''
    
    X_batches, y_batches = [], []
    
    samples_num = len(X_val)
    samples_inds = np.arange(samples_num)
    np.random.shuffle(samples_inds)
    
    cur_x, cur_y = [], []
    
    for i, cur_ind in enumerate(samples_inds):
        if i % batch_sz < batch_sz-1:
            cur_x.append(X_val[cur_ind])
            cur_y.append(y_val[cur_ind]-1)
        else:
            cur_x.append(X_val[cur_ind])
            cur_y.append(y_val[cur_ind]-1)
            X_batches.append(cur_x)
            y_batches.append(cur_y)
            cur_x, cur_y = [], []
          
    X_batches.append(cur_x)
    y_batches.append(cur_y)
    
    return np.array(X_batches), np.array(y_batches)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

def get_score(preds, targets):
        
    accuracy = accuracy_score(targets, preds)
    precision_macro = precision_score(targets, preds, average='macro')
    precision_micro = precision_score(targets, preds, average='micro')
    f1_macro = f1_score(targets, preds, average='macro')
    f1_micro = f1_score(targets, preds, average='micro')
    
    return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

In [0]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity as cos

Подаем в сеть ембеддинг вопроса и ембеддинги вариантов ответов. Сеть содержит отдельные слои для вопроса и отдельные слои для всех вариантов ответов.

In [0]:
class DSSMNet(nn.Module):
    def __init__(self, bow_size):
        super().__init__()
        self.bow_size = bow_size
        self.linear_0_1 = nn.Linear(bow_size, 300)
        self.linear_0_2 = nn.Linear(300, 300)
        self.linear_0_3 = nn.Linear(300, 128)
        
        self.linear_1_1 = nn.Linear(bow_size, 300)
        self.linear_1_2 = nn.Linear(300, 300)
        self.linear_1_3 = nn.Linear(300, 128)    
        
        self.linear_2_1 = nn.Linear(bow_size, 300)
        self.linear_2_2 = nn.Linear(300, 300)
        self.linear_2_3 = nn.Linear(300, 128)
        
        self.linear_3_1 = nn.Linear(bow_size, 300)
        self.linear_3_2 = nn.Linear(300, 300)
        self.linear_3_3 = nn.Linear(300, 128)
        
        self.linear_4_1 = nn.Linear(bow_size, 300)
        self.linear_4_2 = nn.Linear(300, 300)
        self.linear_4_3 = nn.Linear(300, 128)
        
    def forward(self, sample):
        bow_size = self.bow_size
        
        question = sample[:,0,:]
        answer1, answer2, answer3, answer4 = sample[:,1,:], sample[:,2,:], sample[:,3,:], sample[:,4,:]
        
        outputs = torch.Tensor().type(torch.cuda.FloatTensor)
        cosines = torch.Tensor().type(torch.cuda.FloatTensor)
        
        question = torch.tanh(self.linear_0_1(question))
        question = torch.tanh(self.linear_0_2(question))
        question = torch.tanh(self.linear_0_3(question))
        outputs = torch.cat((outputs, question.unsqueeze(0)), dim=0)
        
        answer1 = torch.tanh(self.linear_1_1(answer1))
        answer1 = torch.tanh(self.linear_1_2(answer1))
        answer1 = torch.tanh(self.linear_1_3(answer1))
        outputs = torch.cat((outputs, answer1.unsqueeze(0)), dim=0)
        
        answer2 = torch.tanh(self.linear_2_1(answer2))
        answer2 = torch.tanh(self.linear_2_2(answer2))
        answer2 = torch.tanh(self.linear_2_3(answer2))
        outputs = torch.cat((outputs, answer2.unsqueeze(0)), dim=0)
        
        answer3 = torch.tanh(self.linear_3_1(answer3))
        answer3 = torch.tanh(self.linear_3_2(answer3))
        answer3 = torch.tanh(self.linear_3_3(answer3))
        outputs = torch.cat((outputs, answer3.unsqueeze(0)), dim=0)
        
        answer4 = torch.tanh(self.linear_4_1(answer4))
        answer4 = torch.tanh(self.linear_4_2(answer4))
        answer4 = torch.tanh(self.linear_4_3(answer4))
        outputs = torch.cat((outputs, answer4.unsqueeze(0)), dim=0)
        
        
        for i in range(1,len(outputs)):
            cosines = torch.cat((cosines, F.cosine_similarity(outputs[0], outputs[i], dim=1).unsqueeze(0)))
            
        probs = F.softmax(cosines.view(-1,4))
        
        return probs     

In [0]:
def train(model, optimizer, X_train, y_train, batch_size):
    train_loss, all_preds, all_tests = [], [], []
    model.train()
    X_batches, y_batches = get_batch(X_train, y_train, batch_size)
    
    for batch in zip(X_batches, y_batches):
        X_batch, y_batch = batch
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        X_batch = torch.from_numpy(X_batch).type(torch.cuda.FloatTensor)
        y_batch = torch.from_numpy(y_batch).type(torch.cuda.LongTensor)
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        
        train_loss.append(loss.item())
        
        max_ind = y_pred.max(1, keepdim=True)[1]
        y_pred = max_ind.view(1,-1)[0]
        all_preds.extend(y_pred.detach().cpu().numpy())
        all_tests.extend(y_batch.cpu().numpy())
        
        loss.backward()
        optimizer.step()
        
    score = get_score(np.array(all_preds), np.array(all_tests))
        
    return np.mean(train_loss), list(score)

In [0]:
def validate(model, optimizer, X_test, y_test, batch_size):
    val_loss, all_preds, all_tests = [], [], []
    model.eval()
    X_batches, y_batches = get_batch(X_test, y_test, batch_size)
    
    for batch in zip(X_batches, y_batches):
        X_batch, y_batch = batch
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        X_batch = torch.from_numpy(X_batch).type(torch.cuda.FloatTensor)
        y_batch = torch.from_numpy(y_batch).type(torch.cuda.LongTensor)
        
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        
        val_loss.append(loss.item())
        
        max_ind = y_pred.max(1, keepdim=True)[1]
        y_pred = max_ind.view(1,-1)[0]
        all_preds.extend(y_pred.detach().cpu().numpy())
        all_tests.extend(y_batch.cpu().numpy())
        
    score = get_score(np.array(all_preds), np.array(all_tests))
        
    return np.mean(val_loss), list(score)

In [0]:
from IPython.display import clear_output
def launch_net(model, optimizer, X_train, y_train, X_test, y_test,
               train_batch=64, test_batch=16, epochs=25, graph=False):
    all_train_loss, all_val_loss = [], []
    all_train_acc, all_val_acc = [], []
    all_scores = []
    max_acc = 0

    for epoch in pb(range(epochs)):
        cur_train = train(model, optimizer, X_train, y_train, train_batch)
        cur_val = validate(model, optimizer, X_test, y_test, test_batch)
        all_train_loss.append(cur_train[0])
        all_train_acc.append(cur_train[1][0])
        all_val_loss.append(cur_val[0])
        all_val_acc.append(cur_val[1][0])
        all_scores.append(cur_val[1])
        max_acc = cur_val[1][0] if cur_val[1][0] > max_acc else max_acc
        clear_output()
        print('Epoch {}:'.format(epoch+1))
        print("Train loss: {}".format(cur_train[0]))
        print("Validation loss: {}".format(cur_val[0]))
        print("Train accuracy: {}".format(cur_train[1][0]))
        print("Validation accuracy: {}".format(cur_val[1][0]))
        print("Max validation accuracy: {}".format(max_acc))
        
        if graph:
          f, axarr = plt.subplots(1,2, figsize=(20,10))

          axarr[0].plot(all_train_loss, label='Train loss')
          axarr[0].plot(all_val_loss, label='Val loss')
          axarr[0].grid()
          axarr[0].legend(fontsize=15)
          axarr[0].set_title('Loss after epoch #{}'.format(epoch+1), size=15)

          axarr[1].plot(all_train_acc, label='Train accuracy')
          axarr[1].plot(all_val_acc, label='Val accuracy')
          axarr[1].grid()
          axarr[1].legend(fontsize=15)
          axarr[1].set_title('Accuracy after epoch #{}'.format(epoch+1), size=15)
          plt.show()
        
    all_scores = np.max(np.array(all_scores), axis=0)
    return all_scores

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}

## FastText

In [0]:
embeddings = fasttext_embs
target = np.array(df['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [25]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:52 Time:  0:00:52


Epoch 30:
Train loss: 1.386393184785719
Validation loss: 1.3864986274300553
Train accuracy: 0.24510403916768667
Validation accuracy: 0.24686448455185073
Max validation accuracy: 0.27011318446007954


## ELMO

In [0]:
embeddings = elmo_embs
target = np.array(df['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
epochs = 25

In [28]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (25 of 25) |########################| Elapsed Time: 0:00:51 Time:  0:00:51


Epoch 25:
Train loss: 1.3862954014307494
Validation loss: 1.3863021333043168
Train accuracy: 0.24204406364749081
Validation accuracy: 0.24931171612113795
Max validation accuracy: 0.2627714897522178


## BERT

In [0]:
embeddings = bert_embs
target = np.array(df['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
epochs = 25

In [31]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (25 of 25) |########################| Elapsed Time: 0:00:47 Time:  0:00:47


Epoch 25:
Train loss: 1.3862950221284644
Validation loss: 1.3863451230816726
Train accuracy: 0.24581803345573236
Validation accuracy: 0.25910064239828695
Max validation accuracy: 0.2615478739675742


## RusVectores

In [0]:
embeddings = rusvec_embs
target = np.array(df['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
epochs = 25

In [34]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['RusVectores'] = results

100% (25 of 25) |########################| Elapsed Time: 0:00:43 Time:  0:00:43


Epoch 25:
Train loss: 1.3862735915493656
Validation loss: 1.3863289542314483
Train accuracy: 0.253467972256222
Validation accuracy: 0.2410523095747935
Max validation accuracy: 0.2667482410523096


In [35]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.266748,0.270113,0.262771,0.261548
Precision Macro,0.267006,0.270243,0.263015,0.260641
Precision Micro,0.266748,0.270113,0.262771,0.261548
F1 Macro,0.266540,0.270199,0.262578,0.260611
F1 Micro,0.266748,0.270113,0.262771,0.261548


# История

In [15]:
hist_inds = []

for i in range(len(data)):
  if data.iloc[i,6] == 'История':
    hist_inds.append(i)
len(hist_inds)

2498

In [0]:
elmo_embs_hist = elmo_embs[hist_inds]
bert_embs_hist = bert_embs[hist_inds]
fasttext_embs_hist = fasttext_embs[hist_inds]
rusvec_embs_hist = rusvec_embs[hist_inds]

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
data2 = data[data['label'] == 'История']
epochs = 20

## FastText

In [0]:
embeddings = fasttext_embs_hist
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [22]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


Epoch 30:
Train loss: 1.3858301719029744
Validation loss: 1.389969825744629
Train accuracy: 0.25253603844100375
Validation accuracy: 0.2304
Max validation accuracy: 0.2928


## ELMO

In [0]:
embeddings = elmo_embs_hist
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [25]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:12 Time:  0:00:12


Epoch 30:
Train loss: 1.3859500447909037
Validation loss: 1.3872271567583083
Train accuracy: 0.24719701014415377
Validation accuracy: 0.2192
Max validation accuracy: 0.2768


## BERT

In [0]:
embeddings = bert_embs_hist
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [28]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:11 Time:  0:00:11


Epoch 30:
Train loss: 1.386215047041575
Validation loss: 1.3847931772470474
Train accuracy: 0.24826481580352375
Validation accuracy: 0.2592
Max validation accuracy: 0.28


## RusVectores

In [0]:
embeddings = rusvec_embs_hist
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [31]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['RusVectores'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


Epoch 30:
Train loss: 1.3861871242523194
Validation loss: 1.387191393971443
Train accuracy: 0.2594767752269087
Validation accuracy: 0.2448
Max validation accuracy: 0.2944


In [32]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.294400,0.292800,0.276800,0.280000
Precision Macro,0.293435,0.295615,0.275867,0.280598
Precision Micro,0.294400,0.292800,0.276800,0.280000
F1 Macro,0.294034,0.292644,0.275175,0.279996
F1 Micro,0.294400,0.292800,0.276800,0.280000


# Медицина

In [34]:
med_inds = []

for i in range(len(data)):
  if data.iloc[i,6] == 'Медицина':
    med_inds.append(i)
len(med_inds)

4013

In [0]:
elmo_embs_med = elmo_embs[med_inds]
bert_embs_med = bert_embs[med_inds]
fasttext_embs_med = fasttext_embs[med_inds]
rusvec_embs_med = rusvec_embs[med_inds]

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
data2 = data[data['label'] == 'Медицина']
epochs = 20

## FastText

In [0]:
embeddings = fasttext_embs_med
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [39]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:17 Time:  0:00:17


Epoch 30:
Train loss: 1.38615878423055
Validation loss: 1.3864139337388297
Train accuracy: 0.25390495181123296
Validation accuracy: 0.22609561752988047
Max validation accuracy: 0.28286852589641437


## ELMO

In [0]:
embeddings = elmo_embs_med
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [42]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:19 Time:  0:00:19


Epoch 30:
Train loss: 1.3853320231040318
Validation loss: 1.3862686327525549
Train accuracy: 0.257892987703556
Validation accuracy: 0.24800796812749004
Max validation accuracy: 0.2818725099601594


## BERT

In [0]:
embeddings = bert_embs_med
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [45]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:19 Time:  0:00:19


Epoch 30:
Train loss: 1.3853332921862602
Validation loss: 1.386294378174676
Train accuracy: 0.2535726154868727
Validation accuracy: 0.2340637450199203
Max validation accuracy: 0.30278884462151395


## RusVectores

In [0]:
embeddings = rusvec_embs_med
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [49]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['RusVectores'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:16 Time:  0:00:16


Epoch 30:
Train loss: 1.3829824849963188
Validation loss: 1.386589595249721
Train accuracy: 0.26021934197407776
Validation accuracy: 0.23705179282868527
Max validation accuracy: 0.2739043824701195


In [50]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.273904,0.282869,0.281873,0.302789
Precision Macro,0.275671,0.283622,0.281040,0.301072
Precision Micro,0.273904,0.282869,0.281873,0.302789
F1 Macro,0.272586,0.280716,0.277321,0.295789
F1 Micro,0.273904,0.282869,0.281873,0.302789


## Биология

In [51]:
bio_inds = []

for i in range(len(data)):
  if data.iloc[i,6] == 'Биология':
    bio_inds.append(i)
len(bio_inds)

2184

In [0]:
elmo_embs_bio = elmo_embs[bio_inds]
bert_embs_bio = bert_embs[bio_inds]
fasttext_embs_bio = fasttext_embs[bio_inds]
rusvec_embs_bio = rusvec_embs[bio_inds]

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
data2 = data[data['label'] == 'Биология']
epochs = 20

## FastText

In [0]:
embeddings = fasttext_embs_bio
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [58]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


Epoch 30:
Train loss: 1.3864129644173842
Validation loss: 1.386205073765346
Train accuracy: 0.24236874236874237
Validation accuracy: 0.24175824175824176
Max validation accuracy: 0.2802197802197802


## ELMO

In [0]:
embeddings = elmo_embs_bio
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [61]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


Epoch 30:
Train loss: 1.3862673456852252
Validation loss: 1.386000953401838
Train accuracy: 0.2625152625152625
Validation accuracy: 0.27289377289377287
Max validation accuracy: 0.2948717948717949


## BERT

In [0]:
embeddings = bert_embs_bio
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [64]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


Epoch 30:
Train loss: 1.386308518739847
Validation loss: 1.3858578375407629
Train accuracy: 0.24114774114774115
Validation accuracy: 0.27472527472527475
Max validation accuracy: 0.2893772893772894


## RusVectores

In [0]:
embeddings = rusvec_embs_bio
target = np.array(data2['5'])

X_train, X_test, y_train, y_test = TTS(embeddings, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(embeddings.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [67]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['RusVectores'] = results

100% (30 of 30) |########################| Elapsed Time: 0:00:10 Time:  0:00:10


Epoch 30:
Train loss: 1.3862801790237427
Validation loss: 1.3865962096623012
Train accuracy: 0.2509157509157509
Validation accuracy: 0.24358974358974358
Max validation accuracy: 0.2765567765567766


In [68]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.276557,0.280220,0.294872,0.289377
Precision Macro,0.277054,0.280572,0.297706,0.289367
Precision Micro,0.276557,0.280220,0.294872,0.289377
F1 Macro,0.276782,0.280425,0.294769,0.288787
F1 Micro,0.276557,0.280220,0.294872,0.289377
